# Сбор и разметка данных

## Урок 9. Инструменты разметки наборов данных


In [1]:
import os
import wget

import pandas as pd

from sklearn.model_selection import train_test_split

import nltk
# Загружаем предобученную модель токенизации текста
nltk.download('punkt')

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nusha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Практическое задание

**Задание 1.**
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

Возьмем датасет новостей lenta.ru

In [2]:
df = pd.read_csv('./news_lenta_ru.csv', engine='python', on_bad_lines='skip')
df.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая


**Задание 2.**
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

Загрузим тональный словарь.

In [3]:
url = 'https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv'
file = 'kartaslovsent.csv'
if not os.path.isfile("./kartaslovsent.csv"):
    wget.download(url, out=f"./{file}")

In [4]:
df_tone_dic = pd.read_csv('./kartaslovsent.csv', sep=';', on_bad_lines='skip')
df_tone_dic.head()

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19


Перемешаем и разделим датафрейм на две части (1-разметка вручную, 2-разметка на основе правил).

In [5]:
title1, title2 = train_test_split(df['title'], train_size=5, random_state=42)

In [6]:
df_title1 = pd.DataFrame(title1)

In [7]:
df_title1.head()

,title
5734,Ястржембский называет имя арестованного за взр...
5191,"Чемпионат Италии: ""Ювентус"" ушел в отрыв"
5390,Из Чернокозово отпущены первые амнистированные...
860,Парламент Украины затребовал у Конгресса США п...
7270,Пилот Формулы-1 уцелел в авиакатастрофе


In [8]:
df_title2 = pd.DataFrame(title2)

In [9]:
df_title2.head()

,title
6252,"Чтобы не платить налог, интернет-изданиям нужн..."
4684,Кубинского дипломата выставили из США
1731,Правительство РФ: третье тысячеление наступит ...
4742,Взрывчатку в Сибирь отправили по почте
4521,В Дагестане совершено покушение на генерала по...


In [10]:
# Функция для присвоения метки настроения на основе наличия положительных или отрицательных слов
def get_sentiment(text, df_td=df_tone_dic):
    # Токенизация текста на отдельные слова
    words = nltk.word_tokenize(text.lower())
    sum_value = 0  # Сумма всех зачений весов найденных слов

    for word in words:
        if len(df_td[df_td.term == word]):
            df_filtered = df_td[df_td['term'] == word]
            value = df_filtered['value'].iloc[0]
            sum_value += value

    return sum_value

In [11]:
# Функция для присвоения метки настроения на основе оценки полярности настроения
def get_sentiment_label(score):
    if score < -0.35:
        return 'negative'
    elif score >= 0.55:
        return 'positive'
    else:
        return 'neutral'

In [12]:
print(get_sentiment('Как хорошо, что я успел вовремя'))
print(get_sentiment('Удар молнии'))
print(get_sentiment('Всё нормально'))

1.33
-0.68
0


In [13]:
print(get_sentiment_label(get_sentiment('Как хорошо, что я успел вовремя')))
print(get_sentiment_label(get_sentiment('Удар молнии')))
print(get_sentiment_label(get_sentiment('Всё нормально')))

positive
negative
neutral


Присвоим метки настроения на основе правил.

In [14]:
df_title2['sentiment'] = df_title2['title'].apply(get_sentiment)
df_title2['sentiment_label'] = df_title2['sentiment'].apply(get_sentiment_label)

**Задача 3.**
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

Сохраним датафрем в CSV файл.

In [15]:
df_title1.to_csv('./manual_marking.csv', index=False)

Запускаем локально label-studio и делаем разметку. \
Экспортируем размеченный вручную CSV файл и загружаем его в датафрейм.

In [16]:
df_title1 = pd.read_csv('./project-1-at-2024-05-29-15-11-3822a918.csv', engine='python', on_bad_lines='skip')
df_title1.head()

,annotation_id,annotator,created_at,id,lead_time,sentiment,title,updated_at
0,1,1,2024-05-29T15:08:25.109948Z,1,18.735,positive,Ястржембский называет имя арестованного за взр...,2024-05-29T15:08:33.034497Z
1,2,1,2024-05-29T15:09:05.374325Z,2,13.131,neutral,"Чемпионат Италии: ""Ювентус"" ушел в отрыв",2024-05-29T15:17:06.400265Z
2,3,1,2024-05-29T15:09:26.498394Z,3,6.770,negative,Из Чернокозово отпущены первые амнистированные...,2024-05-29T15:09:26.498394Z
3,4,1,2024-05-29T15:09:59.909605Z,4,7.341,neutral,Парламент Украины затребовал у Конгресса США п...,2024-05-29T15:09:59.909605Z
4,5,1,2024-05-29T15:10:25.863949Z,5,4.486,positive,Пилот Формулы-1 уцелел в авиакатастрофе,2024-05-29T15:10:25.863949Z


In [17]:
df_title1 = df_title1[['title', 'sentiment']].rename(columns={'sentiment': 'sentiment_label'})
df_title1.head()

,title,sentiment_label
0,Ястржембский называет имя арестованного за взр...,positive
1,"Чемпионат Италии: ""Ювентус"" ушел в отрыв",neutral
2,Из Чернокозово отпущены первые амнистированные...,negative
3,Парламент Украины затребовал у Конгресса США п...,neutral
4,Пилот Формулы-1 уцелел в авиакатастрофе,positive


**Задача 4.**
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

In [18]:
df_title = pd.concat([df_title1, df_title2], axis=0).reset_index(drop=True)
df_title.drop(columns=['sentiment'], inplace=True)
df_title.head(10)

,title,sentiment_label
0,Ястржембский называет имя арестованного за взр...,positive
1,"Чемпионат Италии: ""Ювентус"" ушел в отрыв",neutral
2,Из Чернокозово отпущены первые амнистированные...,negative
3,Парламент Украины затребовал у Конгресса США п...,neutral
4,Пилот Формулы-1 уцелел в авиакатастрофе,positive
5,"Чтобы не платить налог, интернет-изданиям нужн...",neutral
6,Кубинского дипломата выставили из США,neutral
7,Правительство РФ: третье тысячеление наступит ...,neutral
8,Взрывчатку в Сибирь отправили по почте,neutral
9,В Дагестане совершено покушение на генерала по...,negative


In [19]:
df_title['sentiment_label'].value_counts()

sentiment_label
neutral     7463
positive    1351
negative    1186
Name: count, dtype: int64

**Задача 5.**
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

In [20]:
le = LabelEncoder()
df_title['sentiment_label'] = le.fit_transform(df_title['sentiment_label'].astype(str))
df_title.head(10)

,title,sentiment_label
0,Ястржембский называет имя арестованного за взр...,2
1,"Чемпионат Италии: ""Ювентус"" ушел в отрыв",1
2,Из Чернокозово отпущены первые амнистированные...,0
3,Парламент Украины затребовал у Конгресса США п...,1
4,Пилот Формулы-1 уцелел в авиакатастрофе,2
5,"Чтобы не платить налог, интернет-изданиям нужн...",1
6,Кубинского дипломата выставили из США,1
7,Правительство РФ: третье тысячеление наступит ...,1
8,Взрывчатку в Сибирь отправили по почте,1
9,В Дагестане совершено покушение на генерала по...,0


In [21]:
# перемешаем данные
df_title = shuffle(df_title)

In [22]:
# разделим датасет на размеченную и неразмеченную выборки
labeled_title, unlabeled_title = train_test_split(df_title, train_size=0.2, random_state=42)

In [23]:
# функция для обучения модели логистической регрессии на размеченных данных
def train_model(labeled_title):
    # Векторизация текстовых данных с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(labeled_title['title'])
    y = labeled_title['sentiment_label']

    # Обучение модели логистической регрессии на размеченных данных
    model = LogisticRegression()
    model.fit(X, y)

    return model, vectorizer

In [24]:
# обучение начальной модели на небольшом наборе
model, vectorizer = train_model(labeled_title)

In [25]:
# использование исходной модели для прогнозирования настроения неразмеченных данных
X_unlabeled = vectorizer.transform(unlabeled_title['title'])
y_unlabeled_predicted = model.predict(X_unlabeled)

In [26]:
# вычисление неопределенности или энтропии предсказаний
y_unlabeled_proba = model.predict_proba(X_unlabeled)
uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)

In [27]:
# выбор 100 наиболее неопределенных точек данных для маркировки человеком
labeled_title_new = unlabeled_title.iloc[uncertainty.argsort()[:100]]
unlabeled_title_new = unlabeled_title.iloc[uncertainty.argsort()[100:]]

In [28]:
# разметка новых точек данных и добавление их к размеченному множеству
labeled_title = pd.concat([labeled_title, labeled_title_new])
labeled_title.shape

(2100, 2)

In [29]:
# переобучение модели на расширенном маркированном множестве
model, vectorizer = train_model(labeled_title)

**Задача 6.**
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.

In [30]:
### Загружаем тестовый датасет
title_test = pd.read_csv('./test.csv', engine='python', on_bad_lines='skip')

In [31]:
title_test['sentiment_label'] = le.fit_transform(title_test['sentiment_label'].astype(str))
title_test.head()

,title,sentiment_label
0,Вагоны московского метро будут складываться га...,1
1,В Чечню пытались проникнуть наемники из Пакистана,1
2,Таллинские заключенные предпочитают голод духоте,0
3,"Адвокатам ""Моста"" не удалось дать отвод следов...",1
4,В поезде Донецк-Москва нашли взрывчатку,1


In [32]:
X_test = vectorizer.transform(title_test['title'])
y_test_predicted = model.predict(X_test)

В качестве критерия качества модели выберем метрику F1-measure. Она отражает точностью (Precision) и полноту (Recall) и рассчитывается как их среднее гармоническое.

$Precision = \cfrac{TP}{TP + FP}$
<br><br>

$Recall = \cfrac{TP}{TP + FN}$
<br><br>

$F_1 = \cfrac{2}{\cfrac{1}{Recall} + \cfrac{1}{Precision}} = 2 \cfrac{Recall \cdot Precision}{Recall + Precision} = \cfrac{TP}{TP + \cfrac{FP + FN}{2}}$

In [33]:
f1 = f1_score(title_test['sentiment_label'], y_test_predicted, average='micro')
f1

0.84068

Эффективность обученной модели составила 84%.

Разметка на основе правил существенно экономит время и увеличивает производительность, но не всегда положительно влияет на качество набора данных.